## Escolhendo Entre RDDs e DataFrames: Uma Decisão Estratégica no Spark

In [9]:
from pyspark import RDD  # Importe a classe RDD do módulo pyspark
from pyspark.sql import SparkSession, DataFrame  # Importe a classe DataFrame do módulo pyspark.sql

# Inicia uma sessão Spark
spark = SparkSession.builder.appName("ProductDataAnalysis").getOrCreate()

# Atribui o SparkContext à variável 'sc'
sc = spark.sparkContext

# Carrega os dados do arquivo CSV em um RDD
file_path = "product+classification+and+clustering/pricerunner_aggregate.csv"

In [11]:
# Criar um objeto RDD
rdd = sc.textFile(file_path)

In [10]:
# Fazendo uma verificação para confirmar se de fato a variável rdd é um objeto do tipo RDD
if isinstance(rdd, RDD):
    print("Estamos lidando com um RDD.")
    print(rdd.toDebugString())
elif isinstance(rdd, DataFrame):
    print("Estamos lidando com um DataFrame.")
    rdd.printSchema()
    rdd.explain(True)


Estamos lidando com um RDD.
b'(2) product+classification+and+clustering/pricerunner_aggregate.csv MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0 []\n |  product+classification+and+clustering/pricerunner_aggregate.csv HadoopRDD[4] at textFile at NativeMethodAccessorImpl.java:0 []'


In [12]:
# Criar um objeto DataFrame
# Utilizando a SparkSession (geralmente acessível através da variável 'spark')
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [13]:
# Fazendo uma verificação para confirmar se de fato a variável df é um objeto do tipo DataFrame
if isinstance(df, RDD):
    print("Estamos lidando com um RDD.")
    print(rdd.toDebugString())
elif isinstance(df, DataFrame):
    print("Estamos lidando com um DataFrame.")
    df.printSchema()
    df.explain(True)

Estamos lidando com um DataFrame.
root
 |-- Product ID: integer (nullable = true)
 |-- Product Title: string (nullable = true)
 |--  Merchant ID: integer (nullable = true)
 |--  Cluster ID: integer (nullable = true)
 |--  Cluster Label: string (nullable = true)
 |--  Category ID: integer (nullable = true)
 |--  Category Label: string (nullable = true)

== Parsed Logical Plan ==
Relation [Product ID#17,Product Title#18, Merchant ID#19, Cluster ID#20, Cluster Label#21, Category ID#22, Category Label#23] csv

== Analyzed Logical Plan ==
Product ID: int, Product Title: string,  Merchant ID: int,  Cluster ID: int,  Cluster Label: string,  Category ID: int,  Category Label: string
Relation [Product ID#17,Product Title#18, Merchant ID#19, Cluster ID#20, Cluster Label#21, Category ID#22, Category Label#23] csv

== Optimized Logical Plan ==
Relation [Product ID#17,Product Title#18, Merchant ID#19, Cluster ID#20, Cluster Label#21, Category ID#22, Category Label#23] csv

== Physical Plan ==
FileS

In [18]:
# contar o número de produtos em cada partição
def process_partition(iterator):
    # Suponha que queremos processar os dados de alguma maneira. 
    # Aqui, apenas contamos os registros em cada partição
    count = 0
    for _ in iterator:
        count += 1
    yield count

# Aplicando a função a cada partição do RDD criado a partir do arquivo CSV
partition_counts = rdd.mapPartitions(process_partition).collect()

print(partition_counts)  # Mostra o número de registros em cada partição


[17686, 17626]


In [19]:
# Definindo a fração da amostra e a semente para reprodutibilidade
fraction = 0.1  # 10% dos dados
seed = 42  # Semente para garantir que a amostra seja a mesma em execuções diferentes

# Criando a amostra aleatória
sampled_rdd = rdd.sample(False, fraction, seed)

# Coletando e imprimindo os resultados
sampled_data = sampled_rdd.collect()
print(sampled_data)  # Imprime uma amostra aleatória dos dados

['Product ID,Product Title, Merchant ID, Cluster ID, Cluster Label, Category ID, Category Label', '3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones', '10,apple iphone 8 plus 64gb space grey,10,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones', '17,apple iphone 8 plus 64gb silver unlocked,17,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones', '18,apple iphone 8 plus 14 cm 5.5 64 gb 12 mp ios 11 silver,18,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones', '32,apple iphone 7 plus 32gb black,8,2,Apple iPhone 7 Plus 32GB,2612,Mobile Phones', '35,iphone 7 plus 32gb black,24,2,Apple iPhone 7 Plus 32GB,2612,Mobile Phones', '58,apple iphone 7 black 4.7 32gb 4g unlocked sim free,15,3,Apple iPhone 7 32GB,2612,Mobile Phones', '79,apple iphone 8 64gb gold,4,4,Apple iPhone 8 64GB,2612,Mobile Phones', '80,apple iphone 8 64gb space grey,8,4,Apple iPhone 8 64GB,2612,Mobile Phones', '85,apple iphone 8 4.7 inch multi touch retina hd display 64

In [20]:
# Função para adicionar 1 para cada elemento (linha) na partição
def seqOp(acc, value):
    # Cada linha é um registro, então adicionamos 1
    return acc + 1

# Função para combinar as contagens de diferentes partições
def combOp(acc1, acc2):
    # Soma as contagens das duas partições
    return acc1 + acc2

# Inicializando o valor de agregação como 0 (nenhum produto contado ainda)
initial_value = 0

# Usando aggregate para contar o número total de linhas no RDD
total_count = rdd.aggregate(initial_value, seqOp, combOp)

print(total_count)  # Imprime o número total de linhas (produtos) no RDD


35312


## Transformações Avançadas em DataFrames

In [27]:
# Exemplo de exclusão de linhas com valores nulos
df_clean = df.na.drop()

# Lista de nomes de colunas conforme o esquema com espaços
col_names = df_clean.schema.names

# Renomear colunas para remover espaços iniciais
for col_name in col_names:
    new_col_name = col_name.strip()  # strip() remove espaços do começo e do fim
    df_clean = df_clean.withColumnRenamed(col_name, new_col_name)

# Filtrar produtos da categoria "Mobile Phones"
df_filtered = df_clean.select("Product Title", "Category Label").where(df_clean["Category Label"] == "Mobile Phones")

df_filtered.show()


+--------------------+--------------+
|       Product Title|Category Label|
+--------------------+--------------+
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple mq8n2b/a ip...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|sim free iphone 8...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|apple iphone 8 pl...| Mobile Phones|
|iphone 8 plus sim...| Mobile Phones|
|iphone 8 plus 64g...| Mobile Phones|
+--------------------+--------------+
only showing top 20 rows



In [29]:
from pyspark.sql.functions import count

# Agrupando os dados por 'Category Label' e contando os produtos em cada categoria
df_grouped = df_clean.groupBy("Category Label").agg(count("Product ID").alias("Total Products in Category"))

# Exibindo o resultado
df_grouped.show()

+----------------+--------------------------+
|  Category Label|Total Products in Category|
+----------------+--------------------------+
|      Microwaves|                      2342|
|         Fridges|                      3584|
|             TVs|                      3564|
|Washing Machines|                      4044|
|        Freezers|                      2212|
| Digital Cameras|                      2697|
|            CPUs|                      3862|
|     Dishwashers|                      3424|
| Fridge Freezers|                      5501|
|   Mobile Phones|                      4081|
+----------------+--------------------------+



In [30]:
from pyspark.sql.functions import avg

# Se o DataFrame tivesse uma coluna 'Price', poderíamos calcular o preço médio por categoria
# df_grouped_with_price = df.groupBy("Category Label").agg(avg("Price").alias("Average Price"))

# Esta instrução agruparia os dados por 'Category Label' e calcularia o preço médio (Average Price) de produtos em cada categoria

In [ ]:
# Apenas para fins de demonstração pois não temos um dataframe df2
# para fazer a junção
# df_joined = df_clean.join(df2, df_clean["Merchant ID"] == df2["Merchant ID"])

In [ ]:
# Apenas para fins de demonstração, pois em nosso dataframe não temos uma coluna de Price
# df_ordered = df_clean.orderBy(df_clean["Price"].desc())

In [32]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Especificação da janela para classificação dentro de cada categoria com base no título do produto
windowSpec = Window.partitionBy("Category Label").orderBy("Product Title")

# Adicionando uma coluna 'Rank' para classificar os produtos dentro de cada categoria
df_with_rank = df_clean.withColumn("Rank", rank().over(windowSpec))

# Exibindo o resultado
df_with_rank.show()

+----------+--------------------+-----------+----------+--------------------+-----------+--------------+----+
|Product ID|       Product Title|Merchant ID|Cluster ID|       Cluster Label|Category ID|Category Label|Rank|
+----------+--------------------+-----------+----------+--------------------+-----------+--------------+----+
|     14613|2 power 2p n140bg...|         45|      6009|Intel Xeon E3-127...|       2615|          CPUs|   1|
|     16910|2 power 2p n156b3...|         45|      6922|AMD A10-7860K 3.6...|       2615|          CPUs|   2|
|     15270|20m intel xeon pr...|         17|      6229|Intel Xeon E5-265...|       2615|          CPUs|   3|
|     14133|7th gen a10 9700 ...|         18|      5890|AMD A10 9700 3.5 ...|       2615|          CPUs|   4|
|     14248|7th gen a8 9600 a...|         18|      5908|AMD A8 9600 3.1 G...|       2615|          CPUs|   5|
|     15991|a series a6 7400k...|         50|      6482|AMD A6-7400K 3.5G...|       2615|          CPUs|   6|
|     1422

In [ ]:
# Se houvesse uma coluna 'Price', a classificação poderia ser feita com base nela
# windowSpec = Window.partitionBy("Category Label").orderBy("Price")
# df_with_rank = df.withColumn("Rank", rank().over(windowSpec))
